In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [ ]:
# def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
#     y = xt[:,0]*np.cos(xt[:,1])
#     return y.reshape(-1,1)    

In [2]:
label = "QCRE_2D_1_atanh_NW_new"

x_full = np.hstack((np.linspace(-0.25,-0.1 - 0.5/500,150),np.linspace(-0.1,0.1,200),np.linspace(0.1+0.5/500,0.25,150))).reshape(-1,1)
y_full = np.hstack((np.linspace(-0.25,-0.1 - 0.5/500,150),np.linspace(-0.1,0.1,200),np.linspace(0.1+0.5/500,0.25,150))).reshape(-1,1)

X_full,Y_full = np.meshgrid(x_full,y_full)

X_full = X_full.flatten('F').reshape(-1,1)
Y_full = Y_full.flatten('F').reshape(-1,1)
  
xy_full = np.hstack((X_full,Y_full))

inside_square_pts = np.logical_and(np.logical_and(X_full < 0.1, X_full > -0.1),np.logical_and(Y_full < 0.1, Y_full > -0.1)).reshape(-1,)  

xy = xy_full[~inside_square_pts,:]

X = xy[:,0].reshape(-1,1)
Y = xy[:,1].reshape(-1,1)

In [3]:
out_bound_pts_1 = (X == -0.25).reshape(-1,)
out_bound_pts_2 = (Y == -0.25).reshape(-1,)
out_bound_pts_3 = (X == 0.25).reshape(-1,)
out_bound_pts_4 = (Y == 0.25).reshape(-1,)

in_bound_pts_5 = (X == -0.1).reshape(-1,)
in_bound_pts_6 = (Y == -0.1).reshape(-1,)
in_bound_pts_7 = (X == 0.1).reshape(-1,)
in_bound_pts_8 = (Y == 0.1).reshape(-1,)

xy_bound_1 = xy[out_bound_pts_1,:]
xy_bound_2 = xy[out_bound_pts_2,:]
xy_bound_3 = xy[out_bound_pts_3,:]
xy_bound_4 = xy[out_bound_pts_4,:]

xy_bound_5 = xy[in_bound_pts_5,:]
xy_bound_6 = xy[in_bound_pts_6,:]
xy_bound_7 = xy[in_bound_pts_7,:]
xy_bound_8 = xy[in_bound_pts_8,:]

xy_out_bound = np.vstack((xy_bound_1,xy_bound_2,xy_bound_3,xy_bound_4))
xy_in_bound = np.vstack((xy_bound_5,xy_bound_6,xy_bound_7,xy_bound_8))

u_bound_out = np.zeros((np.shape(xy_out_bound)[0],1))
u_bound_in = 1000*np.ones((np.shape(xy_in_bound)[0],1))

xy_bound = np.vstack((xy_out_bound,xy_in_bound)) 
u_bound = np.vstack((u_bound_out,u_bound_in))

xy_test_tensor = torch.from_numpy(xy).float().to(device)

lb_xy = xy[0]
ub_xy = xy[-1]

In [ ]:
u_bound

In [4]:
def trainingdata(N_T,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    
    #choose random N_u points for training
    idx = np.random.choice(xy_bound.shape[0], N_T, replace=False) 
    xy_BC = xy_bound[idx,:] #choose indices from  set 'idx' (x,t)
    u_BC = u_bound[idx].reshape(-1,1)      #choose corresponding u


    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
#     x01 = np.array([[0.0,1.0],[0.0,1.0]])
#     sampling = LHS(xlimits=x01,random_state =seed)
#     samples = sampling(N_f)
    
#     xy_coll = lb_xy + (ub_xy - lb_xy)*samples
#     xy_coll = np.vstack((xy_coll, xy_BC))
    idx_coll = np.random.choice(xy.shape[0], N_f, replace=False)
    xy_coll = xy[idx_coll,:]
     # append training points to collocation points 

    return xy_coll, xy_BC, u_BC

In [7]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)   
        
        self.beta = Parameter(torch.ones((100,len(layers)-2)))
        self.beta.requiresGrad = True
        
        self.train_loss = []
        self.test_loss = []
        
        self.beta_val = []
        
        self.n = torch.tensor(1.0)
            
    'foward pass'
    def forward(self,xy):
        if torch.is_tensor(xy) != True:         
            xy = torch.from_numpy(xy)                
        
        ubxy = torch.from_numpy(ub_xy).float().to(device)
        lbxy = torch.from_numpy(lb_xy).float().to(device)
    
                      
        #preprocessing input 
        xy = (xy - lbxy)/(ubxy - lbxy)
        
        #convert to float
        a = xy.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(z) + self.beta[:,i]*z*self.activation(z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC(self,xy,u):
                
        loss_bc = self.loss_function(self.forward(xy), u)
                
        return loss_bc
    
    def loss_PDE(self, xy_coll, f_hat):
        
        g = xy_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_y = autograd.grad(u,g,torch.ones([xy_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_yy = autograd.grad(u_x_y,g,torch.ones(xy_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dx = u_x_t[:,[0]]
        
        d2u_dx2 = u_xx_yy[:,[0]]
        d2u_dy2 = u_xx_yy[:,[1]]    
        

        f = d2u_dx2 + d2u_dy2
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xy_BC,u_BC,xy_coll,f_hat):

        loss_BC = self.loss_BC(xy_BC,u_BC)
        loss_f = self.loss_PDE(xy_coll,f_hat)
        
        loss_val = loss_BC + loss_f
        
        return loss_val
     
    'callable for optimizer'                                       
    def closure(self):
        optimizer.zero_grad()
        
        loss = self.loss(xy_BC, u_BC, xy_coll,f_hat)
        self.train_loss.append(loss.cpu().detach().numpy())
        
        u_pred = self.test(xy_test_tensor)
        #self.test_loss.append(np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))) #Commented because no true values yet
        self.beta_val.append(self.beta.cpu().detach().numpy())
        
        #print(self.iter,"Train Loss",self.train_loss[-1],"Test Loss",self.test_loss[-1])
        print(self.iter,"Train Loss",self.train_loss[-1])
        
        loss.backward()
        self.iter += 1
  

        return loss        
    'test neural network'
    def test(self,xy_test_tensor):
        u_pred = self.forward(xy_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

In [8]:
max_reps = 10

train_loss_full = []
test_loss_full = []
beta_full = []
elapsed_time= np.zeros((max_reps,1))

for reps in range(max_reps):
    print(reps)
    
    torch.manual_seed(reps*36)
    N_T = 500 #Total number of data points for 'y'
    N_f = 10000 #Total number of collocation points 
    
    xy_coll_np_array, xy_BC_np_array, u_BC_np_array = trainingdata(N_T,N_f,(reps)*22)
        
    xy_coll = torch.from_numpy(xy_coll_np_array).float().to(device)
    xy_BC = torch.from_numpy(xy_BC_np_array).float().to(device)
    u_BC = torch.from_numpy(u_BC_np_array).float().to(device)
        
    f_hat = torch.zeros(xy_coll.shape[0],1).to(device)

    #layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers
    layers = np.array([2,100,100,100,100,100,100,100,100,100,1])
    
    PINN = Sequentialmodel(layers)
   
    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())
    
    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=1, 
                              max_iter = 10000, 
                              max_eval = None, 
                              tolerance_grad = -1, 
                              tolerance_change = -1, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')



    start_time = time.time()
        
    optimizer.step(PINN.closure)
    
    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(PINN.train_loss)
    test_loss_full.append(PINN.test_loss)
    elapsed_time[reps] = time.time() - start_time
    beta_full.append(PINN.beta_val)


    print('Training time: %.2f' % (elapsed_time[reps]))
    
mdic = {"train_loss": train_loss_full,"test_loss": test_loss_full,"Time": elapsed_time, "beta": beta_full, "label": label}
savemat(label+'.mat', mdic)

0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=100, bias=True)
    (1): Linear(in_features=100, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): Linear(in_features=100, out_features=100, bias=True)
    (4): Linear(in_features=100, out_features=100, bias=True)
    (5): Linear(in_features=100, out_features=100, bias=True)
    (6): Linear(in_features=100, out_features=100, bias=True)
    (7): Linear(in_features=100, out_features=100, bias=True)
    (8): Linear(in_features=100, out_features=100, bias=True)
    (9): Linear(in_features=100, out_features=1, bias=True)
  )
)
0 Train Loss 480009.12
1 Train Loss 479967.3
2 Train Loss 479783.66
3 Train Loss 478471.2
4 Train Loss 475705.25
5 Train Loss 470863.7
6 Train Loss 448586.6
7 Train Loss 174326080.0
8 Train Loss 972908.0
9 Train Loss 379111.12
10 Train Loss 346642.38
11 Train Loss 333447.25
12 Trai

307 Train Loss 225220.5
308 Train Loss 224621.7
309 Train Loss 223900.97
310 Train Loss 223490.42
311 Train Loss 223326.67
312 Train Loss 223075.12
313 Train Loss 222788.6
314 Train Loss 222468.11
315 Train Loss 222292.64
316 Train Loss 221788.08
317 Train Loss 221520.39
318 Train Loss 221128.44
319 Train Loss 220867.27
320 Train Loss 220813.77
321 Train Loss 220713.84
322 Train Loss 220535.16
323 Train Loss 220357.58
324 Train Loss 220163.11
325 Train Loss 219904.27
326 Train Loss 219659.66
327 Train Loss 219351.31
328 Train Loss 219264.25
329 Train Loss 219143.19
330 Train Loss 219041.58
331 Train Loss 218934.67
332 Train Loss 218805.78
333 Train Loss 218673.77
334 Train Loss 218476.36
335 Train Loss 218335.89
336 Train Loss 218010.48
337 Train Loss 217863.81
338 Train Loss 217763.28
339 Train Loss 217675.22
340 Train Loss 217453.95
341 Train Loss 217197.62
342 Train Loss 217007.0
343 Train Loss 216834.73
344 Train Loss 216734.17
345 Train Loss 216582.0
346 Train Loss 216336.36
347 T

637 Train Loss 209719.61
638 Train Loss 209715.42
639 Train Loss 209712.86
640 Train Loss 209710.02
641 Train Loss 209705.55
642 Train Loss 209700.8
643 Train Loss 209696.62
644 Train Loss 209692.14
645 Train Loss 209689.25
646 Train Loss 209685.2
647 Train Loss 209682.17
648 Train Loss 209679.2
649 Train Loss 209674.45
650 Train Loss 209665.73
651 Train Loss 209658.97
652 Train Loss 209651.94
653 Train Loss 209642.62
654 Train Loss 209639.2
655 Train Loss 209631.3
656 Train Loss 209628.58
657 Train Loss 209623.84
658 Train Loss 209621.72
659 Train Loss 209617.45
660 Train Loss 209612.42
661 Train Loss 209607.55
662 Train Loss 209603.67
663 Train Loss 209599.88
664 Train Loss 209596.27
665 Train Loss 209592.6
666 Train Loss 209589.27
667 Train Loss 209585.84
668 Train Loss 209581.97
669 Train Loss 209578.45
670 Train Loss 209572.77
671 Train Loss 209565.53
672 Train Loss 209556.08
673 Train Loss 209544.94
674 Train Loss 209534.7
675 Train Loss 209526.08
676 Train Loss 209534.34
677 Tra

969 Train Loss 208147.25
970 Train Loss 208140.92
971 Train Loss 208133.56
972 Train Loss 208125.14
973 Train Loss 208119.11
974 Train Loss 208113.61
975 Train Loss 208111.0
976 Train Loss 208110.48
977 Train Loss 208109.2
978 Train Loss 208108.19
979 Train Loss 208103.53
980 Train Loss 208100.44
981 Train Loss 208095.61
982 Train Loss 208088.69
983 Train Loss 208079.55
984 Train Loss 208067.03
985 Train Loss 208062.84
986 Train Loss 208060.7
987 Train Loss 208055.36
988 Train Loss 208050.02
989 Train Loss 208044.22
990 Train Loss 208036.61
991 Train Loss 208029.97
992 Train Loss 208022.72
993 Train Loss 208018.1
994 Train Loss 208013.48
995 Train Loss 208008.27
996 Train Loss 208005.4
997 Train Loss 208000.55
998 Train Loss 207996.53
999 Train Loss 207994.42
1000 Train Loss 207992.77
1001 Train Loss 207989.8
1002 Train Loss 207987.92
1003 Train Loss 207984.98
1004 Train Loss 207980.17
1005 Train Loss 207975.23
1006 Train Loss 207969.98
1007 Train Loss 207963.97
1008 Train Loss 207956.

1290 Train Loss 206482.1
1291 Train Loss 206479.75
1292 Train Loss 206477.03
1293 Train Loss 206473.36
1294 Train Loss 206469.61
1295 Train Loss 206466.58
1296 Train Loss 206462.75
1297 Train Loss 206458.56
1298 Train Loss 206455.44
1299 Train Loss 206451.89
1300 Train Loss 206447.14
1301 Train Loss 206440.12
1302 Train Loss 206439.02
1303 Train Loss 206433.97
1304 Train Loss 206431.97
1305 Train Loss 206429.6
1306 Train Loss 206427.2
1307 Train Loss 206426.3
1308 Train Loss 206423.77
1309 Train Loss 206421.36
1310 Train Loss 206419.14
1311 Train Loss 206414.95
1312 Train Loss 206407.7
1313 Train Loss 206401.77
1314 Train Loss 206394.7
1315 Train Loss 206387.44
1316 Train Loss 206378.97
1317 Train Loss 206371.19
1318 Train Loss 206366.45
1319 Train Loss 206359.27
1320 Train Loss 206357.17
1321 Train Loss 206353.75
1322 Train Loss 206350.55
1323 Train Loss 206346.81
1324 Train Loss 206340.66
1325 Train Loss 206336.53
1326 Train Loss 206329.47
1327 Train Loss 206322.66
1328 Train Loss 20

...

10195 Train Loss 203073.39
10196 Train Loss 203073.47
10197 Train Loss 203073.39
10198 Train Loss 203073.6
10199 Train Loss 203073.47
10200 Train Loss 203073.45
10201 Train Loss 203073.61
10202 Train Loss 203073.39
10203 Train Loss 203073.47
10204 Train Loss 203073.39
10205 Train Loss 203073.6
10206 Train Loss 203073.47
10207 Train Loss 203073.45
10208 Train Loss 203073.61
10209 Train Loss 203073.39
10210 Train Loss 203073.47
10211 Train Loss 203073.39
10212 Train Loss 203073.6
10213 Train Loss 203073.47
10214 Train Loss 203073.45
10215 Train Loss 203073.61
10216 Train Loss 203073.39
10217 Train Loss 203073.47
10218 Train Loss 203073.39
10219 Train Loss 203073.6
10220 Train Loss 203073.47
10221 Train Loss 203073.45
10222 Train Loss 203073.61
10223 Train Loss 203073.39
10224 Train Loss 203073.47
10225 Train Loss 203073.39
10226 Train Loss 203073.6
10227 Train Loss 203073.47
10228 Train Loss 203073.45
10229 Train Loss 203073.61
10230 Train Loss 203073.39
10231 Train Loss 203073.47
10232 

10503 Train Loss 203073.39
10504 Train Loss 203073.47
10505 Train Loss 203073.39
10506 Train Loss 203073.6
10507 Train Loss 203073.47
10508 Train Loss 203073.45
10509 Train Loss 203073.61
10510 Train Loss 203073.39
10511 Train Loss 203073.47
10512 Train Loss 203073.39
10513 Train Loss 203073.6
10514 Train Loss 203073.47
10515 Train Loss 203073.45
10516 Train Loss 203073.61
10517 Train Loss 203073.39
10518 Train Loss 203073.47
10519 Train Loss 203073.39
10520 Train Loss 203073.6
10521 Train Loss 203073.47
10522 Train Loss 203073.45
10523 Train Loss 203073.61
10524 Train Loss 203073.39
10525 Train Loss 203073.47
10526 Train Loss 203073.39
10527 Train Loss 203073.6
10528 Train Loss 203073.47
10529 Train Loss 203073.45
10530 Train Loss 203073.61
10531 Train Loss 203073.39
10532 Train Loss 203073.47
10533 Train Loss 203073.39
10534 Train Loss 203073.6
10535 Train Loss 203073.47
10536 Train Loss 203073.45
10537 Train Loss 203073.61
10538 Train Loss 203073.39
10539 Train Loss 203073.47
10540 

10809 Train Loss 203073.45
10810 Train Loss 203073.61
10811 Train Loss 203073.39
10812 Train Loss 203073.47
10813 Train Loss 203073.39
10814 Train Loss 203073.6
10815 Train Loss 203073.47
10816 Train Loss 203073.45
10817 Train Loss 203073.61
10818 Train Loss 203073.39
10819 Train Loss 203073.47
10820 Train Loss 203073.39
10821 Train Loss 203073.6
10822 Train Loss 203073.47
10823 Train Loss 203073.45
10824 Train Loss 203073.61
10825 Train Loss 203073.39
10826 Train Loss 203073.47
10827 Train Loss 203073.39
10828 Train Loss 203073.6
10829 Train Loss 203073.47
10830 Train Loss 203073.45
10831 Train Loss 203073.61
10832 Train Loss 203073.39
10833 Train Loss 203073.47
10834 Train Loss 203073.39
10835 Train Loss 203073.6
10836 Train Loss 203073.47
10837 Train Loss 203073.45
10838 Train Loss 203073.61
10839 Train Loss 203073.39
10840 Train Loss 203073.47
10841 Train Loss 203073.39
10842 Train Loss 203073.6
10843 Train Loss 203073.47
10844 Train Loss 203073.45
10845 Train Loss 203073.61
10846 

11116 Train Loss 203073.47
11117 Train Loss 203073.45
11118 Train Loss 203073.61
11119 Train Loss 203073.39
11120 Train Loss 203073.47
11121 Train Loss 203073.39
11122 Train Loss 203073.6
11123 Train Loss 203073.47
11124 Train Loss 203073.45
11125 Train Loss 203073.61
11126 Train Loss 203073.39
11127 Train Loss 203073.47
11128 Train Loss 203073.39
11129 Train Loss 203073.6
11130 Train Loss 203073.47
11131 Train Loss 203073.45
11132 Train Loss 203073.61
11133 Train Loss 203073.39
11134 Train Loss 203073.47
11135 Train Loss 203073.39
11136 Train Loss 203073.6
11137 Train Loss 203073.47
11138 Train Loss 203073.45
11139 Train Loss 203073.61
11140 Train Loss 203073.39
11141 Train Loss 203073.47
11142 Train Loss 203073.39
11143 Train Loss 203073.6
11144 Train Loss 203073.47
11145 Train Loss 203073.45
11146 Train Loss 203073.61
11147 Train Loss 203073.39
11148 Train Loss 203073.47
11149 Train Loss 203073.39
11150 Train Loss 203073.6
11151 Train Loss 203073.47
11152 Train Loss 203073.45
11153 

KeyboardInterrupt: 

In [ ]:
210796/256

In [ ]:
u_pred = PINN.test(xy_test_tensor)

In [ ]:
fig, ax = plt.subplots(1,1)
cmap = plt.cm.jet

img3 = ax.imshow(u_pred.reshape(210,1000),vmin = 0,vmax = 1000,cmap = cmap,extent=[0,1,0,1],aspect = 0.75)


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(xy[:,0], xy[:,1], u_pred, marker='*')

In [ ]:
print(torch.__version__)

In [ ]:
from matplotlib import cm
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

surf = ax.plot_surface(xy[:,0], xy[:,1], u_pred, cmap=cm.jet, linewidth=0, antialiased=False)

In [ ]:
np.min(u_pred)

In [ ]:
u_pred.shape

In [ ]:
a = 0
for i in range(10):
    a = a + test_loss_full[i][-1]
print(a/10)

In [ ]:
a = 0
for i in range(10):
    a = a + train_loss_full[i][-1]
print(a/10)